In [ ]:
import pandas as pd 
from matplotlib import pyplot as plt 
import numpy as np 
%matplotlib inline


In [ ]:
df = pd.read_csv("training.csv")
df.sample(5)

In [ ]:
df.drop("customerID", axis = "columns", inplace = True)
df.dtypes